# Network Analysis Overview

This code base provides a robust framework for analyzing graph data stored in GraphML format. It offers a suite of tools to calculate fundamental metrics and uncover structural insights of the network. Here are the key features and metrics it computes:

- **Community Detection**:
  - Leverages algorithms like Leiden or Louvain to identify communities within the graph. This helps in understanding the modular structure and the clusters of nodes that are more densely connected internally.

- **Graph Diameter and Radius**:
  - Calculates the longest shortest path between any two nodes (diameter) and the minimum eccentricity (radius). These metrics help identify the 'spread' of the network and the central points, respectively.

- **Centrality Measures**:
  - Includes degree, closeness, betweenness, and eigenvector centrality. Each metric highlights different aspects of node importance and influence in the network.

- **Articulation Points and Bridges**:
  - Identifies critical nodes and edges whose removal would fragment the network into disconnected components. This is vital for assessing network resilience and vulnerability.

- **Strongly and Weakly Connected Components**:
  - Analyzes the connectivity and reachability within directed graphs, helping to reveal the underlying structure and flow paths in the network.

- **Network Transitivity (Clustering Coefficient)**:
  - Measures the degree to which nodes in a graph tend to cluster together, which is useful for understanding the local node connectivity and potential for network segmentation.

- **Graph Periphery and Center**:
  - Identifies the nodes on the outermost layers (periphery) and the most central nodes (center) of the graph, providing insights into the layout and core structure of the network.

- **Path Length and Eccentricity**:
  - Provides measures of the shortest paths and maximum distances within the graph, offering insights into the efficiency of connectivity and the reach of individual nodes.

- **Spectral Analysis**:
  - Uses eigenvalues and eigenvectors of the adjacency matrix for deeper insights into graph properties and for methods like spectral clustering.



In [30]:
import igraph as ig
import leidenalg as la
import numpy as np
import pandas as pd
import collections
from collections import Counter

In [6]:
g = ig.Graph.Read_GraphML("graphall.graphml")

In [8]:
print(f"Number of vertices: {g.vcount()}")
print(f"Number of edges: {g.ecount()}")


Number of vertices: 13029
Number of edges: 50340


In [10]:
print("Vertex attributes:", g.vertex_attributes())

print("Attributes for the first two nodes:")
for v in g.vs[:1]:  # Adjust slice for more or fewer nodes
    print(f"Node {v.index} attributes:")
    for attribute in g.vertex_attributes():
        print(f"  {attribute}: {v[attribute]}")
    print()  # Adds a newline for better readability

Vertex attributes: ['name', 'alias', 'Total_Channels', 'log_chnlcnt', 'Total_Capacity', 'Formatted_Total_Capacity', 'Capacity_Rank', 'Channel_Count_Rank', 'Capacity_Percentile', 'pub_key', 'community', 'id']
Attributes for the first two nodes:
Node 0 attributes:
  name: 3733632067
  alias: heliacal
  Total_Channels: 1.0
  log_chnlcnt: 0.693147180559945
  Total_Capacity: 6789.0
  Formatted_Total_Capacity: 7k sats
  Capacity_Rank: 13026.0
  Channel_Count_Rank: 13030.0
  Capacity_Percentile: Bottom 60%
  pub_key: 027ccec61f4bf1fafb5156931da6527dc104ec3613dd4f4050161d89dd76ab494c
  community: 0.0
  id: n0



In [2]:


# Assuming 'g' is your igraph Graph object and 'community' is the attribute name
community_membership = g.vs['community']  # This retrieves the community attribute for all vertices
community_counts = Counter(community_membership)  # Counts the frequency of each community

# Printing the community counts
# for community, count in community_counts.items():
#     print(f"Community {community}: {count} nodes")


NameError: name 'g' is not defined

In [34]:
def community_detection(graph):
    """
    Detect communities within the graph using the Leiden algorithm from leidenalg library.
    
    Args:
    - graph (ig.Graph): An igraph graph object.
    
    Returns:
    - list: List of community memberships for each vertex in the graph.
    """
    # Find the partition of the graph using the ModularityVertexPartition which aims to maximize modularity
    partition = la.find_partition(graph, la.ModularityVertexPartition)
    
    # The membership attribute gives us the community index for each vertex
    return partition.membership

def graph_properties(g):
    """
    Compute fundamental graph properties such as diameter, radius, and connectivity components.
    
    Args:
    - g (ig.Graph): Graph object.
    
    Returns:
    - dict: Fundamental graph properties.
    """
    return {
        "diameter": g.diameter(),
        "radius": g.radius(),
        "components": len(g.components(mode='WEAK')),
        "strong_components": len(g.components(mode='STRONG'))
    }

def centrality_measures(g):
    """
    Calculate various centrality measures for the graph.
    
    Args:
    - g (ig.Graph): Graph object.
    
    Returns:
    - dict: Centrality measures.
    """
    return {
        "degree": g.degree(),
        "betweenness": g.betweenness(),
        "closeness": g.closeness(),
        "eigenvector": g.eigenvector_centrality()
    }

def critical_points(g):
    """
    Identify articulation points and bridges in the graph.
    
    Args:
    - g (ig.Graph): Graph object.
    
    Returns:
    - dict: Articulation points and bridges.
    """
    return {
        "articulation_points": g.articulation_points(),
        "bridges": g.bridges()
    }

def clustering_coefficient(g):
    """
    Calculate the transitivity (global clustering coefficient) of the graph.
    
    Args:
    - g (ig.Graph): Graph object.
    
    Returns:
    - float: Transitivity value.
    """
    return g.transitivity_undirected()

def spectral_analysis(g):
    """
    Perform spectral analysis on the graph.
    
    Args:
    - g (ig.Graph): Graph object.
    
    Returns:
    - dict: Eigenvalues and eigenvectors.
    """
    return {
        "eigenvalues": g.eigenvalues(),
        "adjacency_matrix": g.get_adjacency().data
    }

In [44]:

# Perform community detection and print the number of communities
communities = community_detection(g)
print(f"Detected {len(communities)} communities.")

unique_communities = set(communities)

# Print the number of unique communities detected
print(f"Detected {len(unique_communities)} communities.")

community_counts = collections.Counter(communities)

# # Printing the number of nodes in each community
# for community_id, count in community_counts.items():
#     print(f"Community {community_id}: {count} nodes")


Detected 13029 communities.
Detected 88 communities.


In [46]:
# Calculate graph properties
properties = graph_properties(g)
print(f"Diameter: {properties['diameter']}, Radius: {properties['radius']}, Components: {properties['components']}")

Diameter: 9, Radius: 1, Components: 61


In [ ]:
# Calculate centrality measures and print
centrality = centrality_measures(g)
print("Centrality measures calculated:", centrality)



In [12]:
# Identify critical points
critical = critical_points(g)
print(f"Articulation Points: {len(critical['articulation_points'])}, Bridges: {len(critical['bridges'])}")



Articulation Points: 1056, Bridges: 6299


In [ ]:
# Calculate transitivity
transitivity = clustering_coefficient(g)
print(f"Transitivity: {transitivity}")



In [ ]:
# Optionally perform spectral analysis if needed
# spectral = spectral_analysis(g)
# print("Spectral analysis results:", spectral)